# 2.5 Xarrays

Este tutorial ha sido copiado y modificado de los tutoriales del paquete Xarray.

Los Xarrays son matrices multidimensionales ("tensores") que pueden tener varios atributos y dimensiones. La estructura principal es ``DataArray``, el array de N dimensiones que es similar a un ``pandas.Series``. La segunda es ``Dataset``, una base de datos multidimensional en memoria. Es un diccionario como contenedor de ``DataArray``, el equivalente a ``pandas.DataFrame``.

Los Xarrrays pueden leerse desde netCDF y desde Zarr.

Encontrará muchos tutoriales útiles en el proyecto Xarray, como por ejemplo [Esto](https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html). 

[Aquí](https://tutorial.xarray.dev/fundamentals/01_datastructures.html) está el libro tutorial de Xarray que introduce Xarray desde la estructura de datos hasta la visualización. En general, Xarray envuelve Numpy y Pandas y se comporta de manera similar a Pandas. La transición para adoptar Xarray debería ser suave si ya estás familiarizado con Numpy y Pandas.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pooch

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

¿Cuáles son las claves/atributos del conjunto de datos?

Encuentra dos formas de imprimir los valores del atributo ``air``

In [ ]:
ds["air"]

In [ ]:
ds.air

In [ ]:
with xr.set_options(display_style="html"):
    display(ds)

El DataArray tiene una dimensión con nombre

In [ ]:
ds.air.dims

y las coordenadas se guardan en ``.coord``:

In [ ]:
ds.air.coords

Los DataArrays pueden guardar atributos

In [ ]:
ds.air.attrs

Agrega nuevos atributos

In [ ]:
# asigna tus propios atributos!
ds.air.attrs["who_is_awesome"] = "xarray"
ds.air.attrs

Los datos subyacentes son una matriz numpy

In [ ]:
print(type(ds.air.data))
print(ds.air.data)



Cómo extraer datos:

* Indexación por etiquetas con ``.sel``.

* Indexación basada en la posición mediante ``.isel``.

In [ ]:
ds.air.isel(time=1).plot(x="lon")

Observará que la temperatura del aire está en grados Kelvin. Podemos convertirla a Celsius eliminando 273.15 y cambiando los atributos ``units``.

In [ ]:
ds2=ds
ds2['air']=ds['air']-273.15
ds2['air']['units']='degC'

También queremos mostrar las longitudes en dirección oeste eliminando 360$^\circ$.

In [ ]:
ds2.coords["lon"]=ds2.coords["lon"]-360

Muestra el promedio de la temperatura

In [ ]:
ds2.air.mean("time").plot()

In [ ]:
ds2.sel(time="2013-05")

Selecciona los datos entre dos fechas y reduce el tamaño del Xarray

In [ ]:
# demostrar el corte
ds.sel(time=slice("2013-05", "2013-07"))

In [ ]:
# "indexación más próxima en varios puntos"
ds.sel(lon=[240.125-360, 234-360], lat=[40.3, 50.3], method="nearest")

### Computación de alto nivel

* agruparpor : Agrupa y reduce los datos

* remuestrear : Groupby especializado para ejes temporales. Reduzca o aumente la muestra de sus datos.

* rodando : Operar en ventanas móviles de sus datos, por ejemplo, media móvil.

* engrosar: reduce la muestra de sus datos.

* ponderado : Ponderar los datos antes de reducirlos

In [ ]:
# grupos estacionales
ds.groupby("time.season")

In [ ]:
# hacer una media estacional
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean = seasonal_mean.sel(season=["DJF", "MAM", "JJA", "SON"])
seasonal_mean

In [ ]:
# remuestreo a frecuencia mensual
ds.resample(time="M").mean()

In [ ]:
# faceta la media_estacional
seasonal_mean.air.plot(col="season")

Podemos guardar Xarrays en archivos NetCDF y Zarr

In [ ]:
# escribir en netCDF
%timeit ds.to_netcdf("my-example-dataset.nc")
!ls -lh my-example-dataset.nc

In [ ]:

%timeit ds.to_zarr(store="./my-example-dataset.zarr",mode="w")
!du -sh ./my-example-dataset.zarr